In [1]:
from fedn import APIClient
import time
import json
import matplotlib.pyplot as plt
import numpy as np

In [2]:
CONTROLLER_HOST = 'fedn.scaleoutsystems.com/<your-project-name>' 
ACCESS_TOKEN = '<your-access-token>'
client = APIClient(CONTROLLER_HOST,token=ACCESS_TOKEN, secure=True,verify=True)

In [ ]:
client.set_active_package('../mnist-pytorch/package.tgz', 'numpyhelper')
client.set_active_model('../mnist-pytorch/seed.npz')
seed_model = client.get_active_model()

### Defining the objective function

Optuna expects an objective function - the function that evaluates a certain set of hyperparameter values. In this example, we will use the test accuracy as a validation score and we want to maximize it.

For each set of hyperparameter values, each 'trial', we will start a new session using the FEDn Python API. In each session/trial, we will select the model with the highest test accuracy and use that in the Optuna objective function to evaluate the trial.

In [4]:
# Helper function to get the model with the highest test accuracy within a session
def get_highest_test_accuracy_in_session(client, n_rounds):
    best_accuracy = 0
    validations_in_session = client.get_validations()['result'][:n_rounds]
    for validation in validations_in_session:
        val_accuracy = json.loads(validation['data'])['test_accuracy']
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy

    return best_accuracy

In [44]:
import optuna

# Objective function which will be sent to Optuna to evaluate the selection of hyperparameter values
def objective(trial):
    # Number of rounds per session
    n_rounds = 1

    # Suggest hyperparameter priors
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)

    # Set session configurations (from seed model)
    session_config = {
                        "helper": "numpyhelper",
                        # "id": session_id,
                        "aggregator": "fedopt",
                        "aggregator_kwargs": {
                            "serveropt": "adam",
                            "learning_rate": learning_rate,
                            "beta1": 0.9,
                            "beta2": 0.99,
                            "tau": 1e-4
                            },
                        "model_id": seed_model['model'], # Restart from seed model in each new session
                        "rounds": n_rounds
                    }

    # Run session and get session id
    result_fedadam = client.start_session(**session_config)
    session_id = result_fedadam['config']['session_id']
    
    # Wait while the current session is active
    while client.get_session_status(session_id) != 'Finished':
        time.sleep(1)
    
    # Return highest 'test' (validation) accuracy
    return get_highest_test_accuracy_in_session(client=client, n_rounds=n_rounds)

### Creating and running an Optuna study

Here we create an Optuna study. Since we are using the test accuracy for evaluation, we want to maximize the objective function in this example. We pass the objective function defined earlier when calling `study.optimize()` and select the number of trials we want to perform.

**Note:** Each trial starts a session, so the number of sessions is `n_trials`.

In [45]:
# Create an Optuna study
study = optuna.create_study(direction="maximize")

# Optimize hyperparameters
study.optimize(objective, n_trials=100) #19m19s to run 100 trials (session) with one round each
print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

[I 2024-09-06 11:22:59,025] A new study created in memory with name: no-name-b1a1a339-93d1-4420-9006-00a43d546dd9


/var/folders/p9/37n8_h0j3w136cfjm88xkpcr0000gn/T/ipykernel_13870/468742983.py:8: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2024-09-06 11:23:10,169] Trial 0 finished with value: 0.18700000643730164 and parameters: {'learning_rate': 0.01060256463073004}. Best is trial 0 with value: 0.18700000643730164.
[I 2024-09-06 11:23:23,317] Trial 1 finished with value: 0.4269999861717224 and parameters: {'learning_rate': 0.06363319752806389}. Best is trial 1 with value: 0.4269999861717224.
[I 2024-09-06 11:23:34,356] Trial 2 finished with value: 0.2070000022649765 and parameters: {'learning_rate': 0.044054634905094}. Best is trial 1 with value: 0.4269999861717224.
[I 2024-09-06 11:23:44,244] Trial 3 finished with value: 0.3019999861717224 and parameters: {'learning_rate': 0.005457455596524018}. Best is trial 1 with value: 0.426999986

### Visualize Optuna's optimization



In [ ]:
import optuna.visualization as vis

# vis.plot_optimization_history(study)
# vis.plot_param_importances(study)



In [53]:
vis.plot_slice(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
                                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                                   50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
                                   62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
                                   74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
                                   86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                                   98, 99],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5}},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [0.01060256463073004, 0.06363319752806389, 0.044054634905094,
                    0.005457455596524018, 0.0006534358443990889,
                    0.005409564385563341, 0.000296117238409861,
                    0.0001548659658268065, 0.0059773222630647126,
                    0.009610312272318417, 0.07936629680356555, 0.0999017085272383,
                    0.03030693483792421, 0.09839071999176227,
                    0.0018120798841072265, 0.02630581835365611,
                    0.0015862265658428628, 0.018963170798261392,
                    0.05591498203997656, 0.0507252826898167, 0.015598714196031616,
                    0.04981982237560933, 0.04472041885602553, 0.04569663540155219,
                    0.02409255925048435, 0.012153379377141215, 0.06204357797505839,
                    0.0318047942959313, 0.0026889985534921607,
                    0.008406962615909752, 0.07570103253376405,
                    0.059086197752773524, 0.03714901133834086, 0.0646915046763561,
                    0.018225411751837776, 0.060873750215417036,
                    0.0005937941753721039, 0.03794230073789979,
                    0.021776847720301837, 0.012825763536089739,
                    0.006945279111941595, 0.003924318415667345,
                    0.0009962905683287265, 0.007485623216043211,
                    0.08121672594329421, 0.03218747043584, 0.0043209290517303035,
                    0.09007673305976491, 0.00011242006456928975,
                    0.052926437331853586, 0.015080515584708165,
                    0.06571961786071892, 0.06853902191868609, 0.043188154256233874,
                    0.03137506646247659, 0.09815877477219788, 0.024965815660978004,
                    0.0002369246671375477, 0.05402719364497381,
                    0.04024469007474665, 0.0021520115377091278,
                    0.06777601481845777, 0.054552697374482345, 0.07824127973056091,
                    0.062190956331727144, 0.04574533489252647,
                    0.026700039728034024, 0.035572253575668344,
                    0.020947893979189017, 0.09807853814362605, 0.074008662788353,
                    0.006494329870318023, 0.013324381014246824,
                    0.047348523959182987, 0.05213136025170205, 0.0468237789021135,
                    0.009671712975184507, 0.028449834843744366,
                    0.03736743378716483, 0.0011102163493235233,
                 